In [58]:
reset -fs

# Install Packages

In [1]:
%pylab inline
import networkx as nx
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd
import numpy as np
import itertools
import plotly.tools as tls
import pickle
import yaml
import os
credentials = yaml.load(open(os.path.expanduser('plotlyCred.yml')))
plotly.tools.set_credentials_file(credentials.get('username'), api_key=credentials.get('api_key'))

Populating the interactive namespace from numpy and matplotlib


In [25]:
# tls.tools.get_embed('https://plot.ly/~chaudao19/65')

# Cleaning Data

In [3]:
df = pd.read_csv("sftopics.csv",header= None)
df.columns = ["index","topics"]
filter_words = [u'honda-gold-wing-riders',u'museums-galleries',u'thinkorswim',u'simple-homes',u'outdoors',u'midi-production',u'audio',u'mindfulness-meditation',u'art-museums',u'tao',u'parents',u'human-sexuality',u'intimacy',u'oculus-rift',"u'learn-how-to-sew'","u'spiritual-awakening'","u'book-swap'","u'drums'","u'board-games'","u'baking'","u'organic-foods'","u'real-estate-investment-education'","u'book-writing'","u'soccer'", "u'soccer'", "u'plays'", "u'worldreligions'", "u'humanrights'", "u'freethinker'", "u'running'", "u'worldreligions'", "u'romance-novels'", "u'conversazione-in-italiano'", "u'interfaith-and-intercultural-dialogue'", "u'food-waste'", "u'heathenry-or-heathenism-or-heathens'", "u'owasp'", "u'pagan-fellowship'", "u'housing'", "u'music'", "u'cooking'", "u'bible'", "u'health-and-alternative-health'", "u'whitewolf'", "u'whitewolf'", "u'm2m'", "u'nps'", "u'alpine-climbing'", "u'alpine-climbing'", "u'scrum'", "u'weightloss'", "u'social'", "u'fitness'", "u'fun-times'", "u'music-appreciation'"]

In [4]:
def clean_df(df):
    indice = df['topics'].apply(lambda x: type(x) == str)
    df = df[indice]
    df['key'] = df['topics'].apply(lambda x: x[1:-1].split(','))
    df['key'] = df['key'].apply(lambda x: [word.strip(" ") for word in x])
    df['TF'] = df['key'].apply(lambda x: [True if word in filter_words else False for word in x])
    indices = df['TF'].apply(lambda x: True not in x)
    df = df[indices]
    df = df.reset_index()
    df = pd.DataFrame(df['key'])
    return df

In [13]:
# df = clean_df(df);

In [14]:
df.head(5)

,key
0,"[u'seo-search-engine-optimization', u'online-m..."
1,"[u'augmented-reality', u'virtual-worlds', u'oc..."
2,"[u'entrepreneurship', u'technology', u'startup..."
3,"[u'consciousness', u'self-exploration', u'self..."
4,"[u'javascript', u'edtech', u'computer-programm..."


In [15]:
def key_list(df):
    urlkey_unique = df['key'].tolist()
    merged_list = list(itertools.chain(*urlkey_unique))
    return merged_list
def gettuples(df,column):
    urlkey_unique = df[column].tolist()
    merged_list = list(itertools.chain(*urlkey_unique))
    return merged_list

# Fitting Data to Plotly Graph

In [16]:
#Initialize the graph
G= nx.Graph()

In [17]:
# Nodes: Unique entity (listofuniqueitmes)
# Edges: Connection between nodes (listoftuplesbetween2nodes)
df['edges'] = df['key'].apply(lambda x: [(a,b) for a in x for b in x if a != b])
nodes = list(set(key_list(df[:100])))
G.add_nodes_from(nodes)
edges = gettuples(df[:100],"edges")
G.add_edges_from(edges)

In [18]:
# plt.figure(figsize=(40,30))
# nx.draw(G,node_color='c',edge_color='k', with_labels=True);

In [19]:
# pos: assign x,y coordinates for each node
pos= nx.spring_layout(G)
G.node = pos
dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d
        
p=nx.single_source_shortest_path_length(G,ncenter)

In [20]:
edge_trace = Scatter(
    x=[], 
    y=[], 
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')
for edge in G.edges():
    x0, y0 = G.node[edge[0]]
    x1, y1 = G.node[edge[1]]
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]
node_trace = Scatter(
    x=[], 
    y=[], 
    text=[],
    mode='markers', 
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[], 
        size=10,         
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    x, y = G.node[node]
    node_trace['x'].append(x)
    node_trace['y'].append(y)
    node_trace['text'].append(node)

In [21]:
for node, adjacencies in enumerate(G.adjacency_list()):
    node_trace['marker']['color'].append(len(adjacencies))
    node_info = '# of connections: '+str(len(adjacencies))
    node_trace['text'].append(node_info)

In [26]:
fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title='<br>Topic Network - San Francisco',
                titlefont=dict(size=16),
                showlegend=True, 
                width=800,
                height=650,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a Chau Dao a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=True),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=True)))

py.iplot(fig, filename='SFGroupsTopics')

In [57]:
fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title='<br>Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=True, 
                width=650,
                height=650,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=True),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=True)))

py.iplot(fig, filename='networkx')